In [ ]:
#Notebook to deduce transmissionn characteristics of stacks of 
#dichroics and filters such as those used in OMX style 
# microscopes. 
#


In [ ]:
import numpy as N
from PIL import Image
import matplotlib.pyplot as plt
import csv
%matplotlib inline 

In [ ]:
def transmit(dye,filterlist):
    sequence=N.zeros((len(filterlist)+1,501))
    sequence[0]=specs[filters.index(dye)]
    startIntensity=sequence[0].sum()
    for i in range(len(filterlist)):
        (filter,mode) = filterlist[i]
        fspectra=specs[filters.index(filter)]
        if(mode == 'T'):
            sequence[i+1]=sequence[i]*(1.0-fspectra)
        else:
            sequence[i+1]=sequence[i]*(fspectra)
    endIntensity=sequence[-1].sum()
    transmit=endIntensity/startIntensity
    return(transmit,sequence)
    

In [ ]:
#Read in filters, dyes and dichroics
#Note, dichroics are reflectance and emission filters are transmission!
filters=['EM-482-35','EM-525-50wo514','EM-629-53','EM-645-75-m','EM-642-80-m','EM-550-49',
         'EM-442-46','EM-525-50','EM-615-24','EM-684-40',
         'R-0039','R-0054','R-0056','R-0048','R-0053','R-0052','R-0058','R-0063',
         'GFP','mCherry','mKate2','mRFP','mScarlet']

#import mciroscope configs from channelConfig file
import spectra.channelConfig
#print spectra.channelConfig.Channels

specs=N.zeros((len(filters),800-300+1))
headers=[None]*len(filters)
numspectra=0
for fname in filters:
    with open('spectra/'+filters[numspectra]+'.csv', 'rUb') as csvfile:
        readvalues=N.zeros((1000,2))
        pointreader = csv.reader(csvfile, delimiter=',')
        i=0
        for row in pointreader:
            if i==0:
                headers[numspectra]=row
            else:
                readvalues[i-1]=[float(row[0]),float(row[1])]
            i=i+1
        #spectra defined in range 300-800, interpolated to 1nm resolution. 
        specs[numspectra]=N.interp(range(300,801,1),readvalues[:i-1,0],readvalues[:i-1,1])
    numspectra=numspectra+1

In [ ]:
microscope='OMX_V3'
channel='Std_GFP'
dye='mKate2'
scopeDef= [x for x in spectra.channelConfig.Channels if (x[0]==microscope and x[1]==channel)]
(efficency,output)=transmit(dye,scopeDef[0][2])
print "Efficency=%2.2f%%" % (efficency*100.)
plt.plot(range(300,801,1),output[0])
plt.plot(range(300,801,1),output[-1])

In [ ]:
#example to build a spectra of mKate2 on OMX V3
# standard drawer:
#primary dichroic 1 = 0039
#primary dichroic 2 = 0063
#secondardy dichroic 1 = 0058 
#emmissionR = 615/24
#emissionT = 442/46
#secondary Dichroic 2 = 0054
#emmissionR = 685/40
#emissionT = 525/50

#Standard drawer mKate2 emission:
#trans primary 2, reflect primary 1, reflect secodnary 1, em 615/24


#example to build a spectra of mKate2 on OMX V3
# standard drawer:
#primary dichroic 1 = 0048
#primary dichroic 2 = 0056
#secondardy dichroic 1 = 0053
#emmissionR = 482/35
#emissionT = 629/59
#secondary Dichroic 2 = 0052 - has been removed in OMX v3 drawer
#emmissionT = 550/49
#emissionR = DIC (650+?)  (but not accessible as dichroic removed)

#Live drawer mKate2 emission:
#trans primary 2, reflect primary 1, trans secodnary 1, em 629/53

In [ ]:
stages=[('R-0056','Trans'),('R-0048','Reflect'),('R-0053','Trans'),('EM-629-53','Reflect')]
start=spectra[filters.index('mKate2')]
startint=start.sum()
print startint
plt.plot(range(400,801,1),start[100:])

for i in range(len(stages)):
    (filter,mode) = stages[i]
    fspectra=spectra[filters.index(filter)]
    if(mode == 'Trans'):
        start=start*(1.0-fspectra)
    elif (mode== 'Reflect'):
        start=start*(fspectra)

    plt.plot(range(400,801,1),start[100:])

endint=start.sum()
print ("Precentate transmission = %f" % ((endint*100.)/startint))
#print filters[i]
#plt.plot(range(400,801,1),spectra[i,100:])


In [ ]:
stages=[['R-0056','T'],['R-0048','R'],['R-0053','T'],['EM-629-53','R']]
dye=spectra[filters.index('mKate2')]
startint=dye.sum()

output=transmit(dye,stages)

plt.plot(range(400,801,1),output[1,100:])

#print output[0]
endint=output[-1,:].sum()

In [ ]:
stages=N.array([['R-0056','T'],['R-0048','R'],['R-0053','T'],['EM-629-53','R']])
    input=[[]]
for i in range (len(stages)):
    input.append()

In [ ]:
#GFP on V3 live cell drawer
stages=[('R-0056','Reflect'),('EM-550-49','Reflect')]
start=spectra[filters.index('GFP')]
startint=start.sum()
plt.plot(range(400,801,1),start[100:])

for i in range(len(stages)):
    (filter,mode) = stages[i]
    fspectra=spectra[filters.index(filter)]
    if(mode == 'Trans'):
        start=start*(1.0-fspectra)
    elif (mode== 'Reflect'):
        start=start*(fspectra)

    plt.plot(range(400,801,1),start[100:])

endint=start.sum()
print ("Precentate transmission = %f" % ((endint*100.)/startint))

In [ ]:
#GFP on V3 standard drawer
stages=[('R-0063','Reflect'),('R-0054','Trans'),('EM-525-50','Reflect')]
start=spectra[filters.index('GFP')]
startint=start.sum()
plt.plot(range(400,801,1),start[100:])

for i in range(len(stages)):
    (filter,mode) = stages[i]
    fspectra=spectra[filters.index(filter)]
    if(mode == 'Trans'):
        start=start*(1.0-fspectra)
    elif (mode== 'Reflect'):
        start=start*(fspectra)

    plt.plot(range(400,801,1),start[100:])

endint=start.sum()
print ("Precentate transmission = %f" % ((endint*100.)/startint))

In [ ]:
#mRFP on V3 Live Cell.
stages=[('R-0056','Trans'),('R-0048','Reflect'),('R-0053','Trans'),('EM-629-53','Reflect')]
start=spectra[filters.index('mScarlet')]
startint=start.sum()
plt.plot(range(400,801,1),start[100:])

for i in range(len(stages)):
    (filter,mode) = stages[i]
    fspectra=spectra[filters.index(filter)]
    if(mode == 'Trans'):
        start=start*(1.0-fspectra)
    elif (mode== 'Reflect'):
        start=start*(fspectra)

    plt.plot(range(400,801,1),start[100:])

endint=start.sum()
print ("Precentate transmission = %f" % ((endint*100.)/startint))

In [ ]:
#mRFP on V3 Standrad.
stages=[('R-0063','Trans'),('R-0039','Reflect'),('R-0058','Reflect'),('EM-615-24','Reflect')]
start=spectra[filters.index('mScarlet')]
startint=start.sum()
plt.plot(range(400,801,1),start[100:])

for i in range(len(stages)):
    (filter,mode) = stages[i]
    fspectra=spectra[filters.index(filter)]
    if(mode == 'Trans'):
        start=start*(1.0-fspectra)
    elif (mode== 'Reflect'):
        start=start*(fspectra)

    plt.plot(range(400,801,1),start[100:])

endint=start.sum()
print ("Precentage transmission = %f" % ((endint*100.)/startint))